# 各谱系固定氨基酸收集与整理

In [1]:
#-*-coding:utf-8-*-
import json
import numpy as np
import random
from tqdm import tqdm
import math 
import re
import pandas as pd
import os 
import datetime
from multiprocessing import Pool
from matplotlib import pyplot as plt

In [ ]:
####各谱系固定氨基酸（包括缺失）在aa_fixed.xlsx文件中
os.chdir('/home/dell-none/Desktop/0426jiqun')
df=pd.read_csv('slope_all_SSR.csv') 

##提取表格信息
xls=pd.read_excel('aa_fixed.xlsx',sheet_name=0).iloc[:5,2:]
list(enumerate(xls.columns))


###以下提取'Alpha','Beta','Gamma','Delta'四种lineage的突变信息
index=[0,6,7,9,10,11,12,13,14,15,16,17,18];new=[]
for i in index:
    df=xls.iloc[:,i]
    new.append(df)
all_info=pd.concat(new,axis=1)

###
dic={1:266,
     2:13465,
     3:21560,
     4:25390,
     5:26242,
     6:26520,
     7:27219,
     8:27391,
     9:27753,
     10:27891,
     11:28271}

label={1:'ORF1a',
     2:'ORF1b',
     3:'S',
     4:'ORF3a',
     5:'E',
     6:'M',
     7:'ORF6',
     8:'ORF7a',
     9:'ORF7b',
     10:'ORF8',
     11:'N'}

import re
def replace(n,j):
    pat1,pat2='\d+','[A-Z]'
    st=dic[n]+int(re.findall(pat1,j)[0])*3-2
    en=dic[n]+int(re.findall(pat1,j)[0])*3+3-2
    lb=label[n]
    
    
    if len(re.findall(pat2,j))==2:
    
        aa=re.findall(pat2,j)[0]+' to '+re.findall(pat2,j)[1]
    
    if len(re.findall(pat2,j))==1 and j[0]==re.findall(pat2,j)[0]:
        aa=re.findall(pat2,j)[0]+' to'
        
    if len(re.findall(pat2,j))==1 and j[-1]==re.findall(pat2,j)[0]:
        aa='to '+re.findall(pat2,j)[0]
    
    
    return [st,en,aa,j,lb]

def get_lst(n):
    evey_co=[]
    
    for i in list(all_info.iloc[:,n]):
        
        
        

        mutate_lst=i.strip().split('，')

#         print(mutate_lst)

        mutate_lst=[j for j in mutate_lst if 'del' not in j]
        
#         print(mutate_lst)
        
        tran=[]
        
        for j in mutate_lst:
            try:
                
                tran.append(replace(n,j))
                
            
            except:
                tran.append('NA')

        evey_co.append(tran)
        
#         print(evey_co)
    
    return evey_co

after_tran=[get_lst(i) for i in range(1,12)]
###上述得到结果和位置
lin=['Alpha','Beta','Gamma','Delta']

all_key_mut=[]

for i in after_tran:
    n=0
    for j in i[:4]:
        lineage=lin[n]
        for k in j:
            if k!='NA':
#                 print(k)
                k.append(lineage)

                all_key_mut.append(k)
        
        n+=1
df=pd.DataFrame(all_key_mut)
df.columns=['ncbi_start','ncbi_end','substitutions','substitutions2','region','lineage']

In [ ]:
#-----------------------------------------------------------------------------
##匹配全位点
#df: 重要位点的dataframe;ssr:所有位点ssr的dataframe
dic1=df[["ncbi_start", "substitutions2"]].set_index("ncbi_start").to_dict(orient='dict')["substitutions2"]

ty,label=[],[]
ssr=pd.read_csv('slope_all_SSR.csv')
for i in list(ssr['orign_position']):
    if i in list(df['ncbi_start']):
        ty.append("Key mutation")
        label.append(dic1[i])
    else:
        ty.append("Not key mutation")
        label.append('NA')
    
ssr['type'],ssr['label']=ty,label

ssr.to_csv('key_aa_mutation_for_ssr.csv',index=False,sep=',')

# 划分区域，进行统计学检验

In [ ]:
##三个值划分成四个区域，然后进行统计学检验

upper=np.percentile(ssr['SSR'],90);lower=np.percentile(ssr['SSR'],10)
median=np.median(ssr['SSR'])


###以Omicron为例
###
mutation=pd.read_excel('Omicron_mutation.xlsx',sheet_name=0)
dic1=df[["ncbi_pos", "mutation"]].set_index("ncbi_pos").to_dict(orient='dict')["mutation"]

ty,label=[],[]
ssr=pd.read_csv('slope_all_SSR.csv')
for i in list(ssr['orign_position']):
    if i in list(df['ncbi_pos']):
        ty.append("Key mutation")
        label.append(dic1[i])
    else:
        ty.append("Not key mutation")
        label.append('NA')
    
ssr['type'],ssr['label']=ty,label

ssr.to_csv('key_aa_mutation_for_omicron.csv',index=False,sep=',')
##Omicron株突变情况
omicron=pd.read_csv('key_aa_mutation_for_omicron.csv')

##区分在 h,med_h,med_l,l四个区域
def get_area(df):
    h=df[df['SSR']>upper].reset_index(drop=True)
    med_l=df[(df['SSR']>lower)&(df['SSR']<median)].reset_index(drop=True)
    med_h=df[(df['SSR']>median)&(df['SSR']<upper)].reset_index(drop=True)
    l=df[df['SSR']<lower].reset_index(drop=True)
    
    return h,med_l,med_h,l
h,med_l,med_h,l=get_area(omicron)

##查看在 med_h以及med_l上有固定氨基酸以及没有固定氨基酸的数量
def get_length(df):
    n=len(df[df['type']=='Not key mutation'])
    m=len(df[df['type']=='Key mutation'])
    
    return n,m
ml_n,ml_m=get_length(med_l)
mh_n,mh_m=get_length(med_h)

###进行校正卡方检验
print(ml_m,ml_n)
print(mh_m,mh_n)
d2 = np.array([[mh_m,mh_n], [ml_m,ml_n]])
chi2_contingency(d2, correction = True)

In [ ]:
###或将aa固定位点先整理好再匹配上去，例如Omicron_fixed_mutation.csv文件格式
import numpy as np
from scipy.stats import chi2_contingency

def get_length(df):
    n=len(df[df['type']=='Not key mutation'])
    m=len(df[df['type']=='Key mutation'])

    return n,m


def cal_p(df):

    dic1=df[["ncbi_pos", "Mutation"]].set_index("ncbi_pos").to_dict(orient='dict')["Mutation"]

    ty,label=[],[]
    ssr=pd.read_csv('slope_all_SSR.csv')
    for i in list(ssr['orign_position']):
        if i in list(df['ncbi_pos']):
            ty.append("Key mutation")
            label.append(dic1[i])
        else:
            ty.append("Not key mutation")
            label.append('NA')

    ssr['type'],ssr['label']=ty,label

    ##三个值划分成四个区域，然后进行统计学检验
    upper=np.percentile(ssr['SSR'],90);lower=np.percentile(ssr['SSR'],10)
    median=np.median(ssr['SSR'])
    print(median)

    h=ssr[ssr['SSR']>upper].reset_index(drop=True)
    med_l=ssr[(ssr['SSR']>lower)&(ssr['SSR']<median)].reset_index(drop=True)
    med_h=ssr[(ssr['SSR']>median)&(ssr['SSR']<upper)].reset_index(drop=True)
    l=ssr[ssr['SSR']<lower].reset_index(drop=True)


   
   
    ml_n,ml_m=get_length(med_l)
    mh_n,mh_m=get_length(med_h)

#     print(ml_m,ml_n)
#     print(mh_m,mh_n)
    d2 = np.array([[mh_m,mh_n], [ml_m,ml_n]])
    chi=chi2_contingency(d2, correction = True)
    
    return chi,d2,ssr


df=pd.read_csv('Omicron_fixed_mutation.csv')
for k,v in df.groupby('Omicron'):
    print(len(ssr))
    chi,d2,ssr=cal_p(v)
    
    print(k)
    
    print(chi)
    
    print(d2)
    
    ssr.to_csv('Omicron_key_ssr_%s.csv'%k,index=False,sep=',')

# nextstrain diversity比较

In [ ]:
###数据从nextstrain上下载，得到四个文件aa/nt_entropy/event.csv

os.chdir('/home/dell-none/Desktop/traditionnal_method')

aa_entropy=pd.read_csv("aa_entropy.tsv",sep='\t')
aa_event=pd.read_csv("aa_event.tsv",sep='\t')

###氨基酸位置改成ncbi核苷酸起始位置
cut_pos=[0,13202,21287,25106,25931,26156,26822,27005,27368,27497,27860]
values = ['ORF1a','ORF1b', 'S', 'ORF3a','E', 'M','ORF6','ORF7a','ORF7b','ORF8','N']
ncbi=[265,13466,21561,25391,26243,26521,27200,27392,27754,27892,28272]

dic_index=dict(zip(values,ncbi))

###转换为NCBI对应的位置
def tran_pos(df):
    new=[]
    alst=list(df['gene'])
    blst=list(df['aa_position'])
    for i in range(len(blst)):
        num=blst[i]
        cds=alst[i]
        
        new.append(dic_index[cds]+(num-1)*3)
    df['ncbi_pos']=new
    return df


aa_entropy2=tran_pos(aa_entropy)
aa_event2=tran_pos(aa_event)

def to_csv(df,filename):
    df.to_csv(filename,index=False,sep=',')
    
to_csv(aa_entropy2,"aa_entropy.csv")
to_csv(aa_event2,"aa_event.csv")